![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2F08+-+R%2Fdepricated&file=08b_gcs+Script+-+Vertex+AI+Custom+Model+-+R+-+Notebook+to+Script.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/08%20-%20R/depricated/08b_gcs%20Script%20-%20Vertex%20AI%20Custom%20Model%20-%20R%20-%20Notebook%20to%20Script.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2F08%2520-%2520R%2Fdepricated%2F08b_gcs%2520Script%2520-%2520Vertex%2520AI%2520Custom%2520Model%2520-%2520R%2520-%2520Notebook%2520to%2520Script.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/08%20-%20R/depricated/08b_gcs%20Script%20-%20Vertex%20AI%20Custom%20Model%20-%20R%20-%20Notebook%20to%20Script.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/08%20-%20R/depricated/08b_gcs%20Script%20-%20Vertex%20AI%20Custom%20Model%20-%20R%20-%20Notebook%20to%20Script.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# 08b_gcs Script - Vertex AI Custom Model - R - From Notebook to Script

**IN ACTIVE DEVELOPMENT - NOT COMPLETE**
    
Dev Notes:
- Python Kernel:
    - use notebook instance with R kernel
    - or, install R here with needed libraries
    
Workflow:
- Construct R script for creating model with annotations - based on code in `08` while leverage read from GCS Fuse (see Resources below)
- Test R script at command line

**Resources:**
- Notebooks:
    - [08 - Vertex AI Custom Model - R - in Notebook](./08%20-%20Vertex%20AI%20Custom%20Model%20-%20R%20-%20in%20Notebook.ipynb)
    - [08Tools - GCS FUSE](./08Tools%20-%20GCS%20FUSE.ipynb)

**Prerequisites:**
- [01 - BigQuery - Table Data Source](../01%20-%20Data%20Sources/01%20-%20BigQuery%20-%20Table%20Data%20Source.ipynb)


---
## Setup

inputs:

In [41]:
SCRIPT_NAME = 'train_gcs.R'

In [42]:
SCRIPT_PATH = './code/' + SCRIPT_NAME

packages:

In [43]:
import os
from IPython.display import Markdown as md

environment:

In [44]:
if os.path.exists('code'):
    print('The code directory alredy exists')
else:
    print('Creating the code directory')
    os.makedirs('code')

The code directory alredy exists


---
## Build the ML Training Code
Use the cell magic `%%writefile` to create a single ML training code file named `train.R`.

### Import Libraries

In [45]:
%%writefile {SCRIPT_PATH}

# library import
library(bigrquery)
library(tidyverse)
library(data.table)

Overwriting ./code/train_gcs.R


### Parse Command Line Inputs

In [46]:
%%writefile -a {SCRIPT_PATH}

# inputs
args <- commandArgs(trailingOnly = TRUE)
project_id <- args[1]
region <- args[2]
experiment <- args[3]
series <- args[4]
bq_project <- args[5]
bq_dataset <- args[6]
bq_table <- args[7]
var_target <- args[8]
var_omit <- args[9]

Appending to ./code/train_gcs.R


### Export Data From BigQuery to GCS

In [47]:
%%writefile -a {SCRIPT_PATH}

# export from BigQuery to GCS
query <- "
EXPORT DATA
    OPTIONS (
        uri = 'gs://%sdata/%s/*.csv',
        format = 'CSV',
        overwrite = true,
        header = true,
        field_delimiter = ','
    )
AS (
    SELECT * EXCEPT(%s, splits)
    FROM `%s.%s.%s`
    WHERE splits = '%s'
)"
#cat(query)

get_data <- function(split){
    # populate export query
    path <- Sys.getenv('AIP_MODEL_DIR')
    path <- sub('gs://', '', path)
    
    split_query <- sprintf(query, path, split, var_omit, bq_project, bq_dataset, bq_table, split)
    cat(split_query)
    
    # run bq export
    export <- bq_perform_query(split_query, billing = bq_project)
    bq_job_wait(export)
    
    # make list of exports files
    files <- list.files(
        path = sprintf('/gcs/%sdata/%s', path, split),
        pattern = '*.csv',
        full.names = TRUE
    )
    
    # read bq export into dataframe from GCS using FUSE
    df <- 
        files %>%
        map_df(~fread(.))
    
    # delete exported copy in GCS
    sapply(files, unlink)
    
    # return the dataframe
    return(df)
}
train <- get_data('TRAIN')
test <- get_data('TEST')

Appending to ./code/train_gcs.R


### Fit Model

In [48]:
%%writefile -a {SCRIPT_PATH}

# logistic regression model
model <- glm(
    Class ~ .,
    data = train,
    family = binomial)

Appending to ./code/train_gcs.R


### Predictions for Test Split

In [49]:
%%writefile -a {SCRIPT_PATH}

# predictions for evaluation
preds <- predict(model, test, type = "response")

Appending to ./code/train_gcs.R


### Evaluate The Model

In [50]:
%%writefile -a {SCRIPT_PATH}

# evaluate
actual <- test[, get(var_target)]
names(actual) <- 'actual'
pred <- tibble(round(preds))
names(pred) <- 'pred'
results <- cbind(actual, pred)
cm <- table(results)

Appending to ./code/train_gcs.R


### Save The Model

In [51]:
%%writefile -a {SCRIPT_PATH}

# save model to file
saveRDS(model, "model.rds")

Appending to ./code/train_gcs.R


### Copy Model To GCS

In [52]:
%%writefile -a {SCRIPT_PATH}

# use Vertex AI Training Pre-Defined Environment Variables to Write to GCS
system2('gsutil', c('cp', 'model.rds', Sys.getenv('AIP_MODEL_DIR')))

Appending to ./code/train_gcs.R


---
## Review The Training Code
Read in the completed script and print it out for review below:

In [53]:
with open(SCRIPT_PATH, 'r') as file:
    data = file.read()
md(f"```r\n\n{data}\n```")

```r


# library import
library(bigrquery)
library(tidyverse)
library(data.table)

# inputs
args <- commandArgs(trailingOnly = TRUE)
project_id <- args[1]
region <- args[2]
experiment <- args[3]
series <- args[4]
bq_project <- args[5]
bq_dataset <- args[6]
bq_table <- args[7]
var_target <- args[8]
var_omit <- args[9]

# export from BigQuery to GCS
query <- "
EXPORT DATA
    OPTIONS (
        uri = 'gs://%sdata/%s/*.csv',
        format = 'CSV',
        overwrite = true,
        header = true,
        field_delimiter = ','
    )
AS (
    SELECT * EXCEPT(%s, splits)
    FROM `%s.%s.%s`
    WHERE splits = '%s'
)"
#cat(query)

get_data <- function(split){
    # populate export query
    path <- Sys.getenv('AIP_MODEL_DIR')
    path <- sub('gs://', '', path)
    
    split_query <- sprintf(query, path, split, var_omit, bq_project, bq_dataset, bq_table, split)
    cat(split_query)
    
    # run bq export
    export <- bq_perform_query(split_query, billing = bq_project)
    bq_job_wait(export)
    
    # make list of exports files
    files <- list.files(
        path = sprintf('/gcs/%sdata/%s', path, split),
        pattern = '*.csv',
        full.names = TRUE
    )
    
    # read bq export into dataframe from GCS using FUSE
    df <- 
        files %>%
        map_df(~fread(.))
    
    # delete exported copy in GCS
    sapply(files, unlink)
    
    # return the dataframe
    return(df)
}
train <- get_data('TRAIN')
test <- get_data('TEST')

# logistic regression model
model <- glm(
    Class ~ .,
    data = train,
    family = binomial)

# predictions for evaluation
preds <- predict(model, test, type = "response")

# evaluate
actual <- test[, get(var_target)]
names(actual) <- 'actual'
pred <- tibble(round(preds))
names(pred) <- 'pred'
results <- cbind(actual, pred)
cm <- table(results)

# save model to file
saveRDS(model, "model.rds")

# use Vertex AI Training Pre-Defined Environment Variables to Write to GCS
system2('gsutil', c('cp', 'model.rds', Sys.getenv('AIP_MODEL_DIR')))

```